In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
# from lstm_model import RNN_LoadForecastser
from lstm_model_2 import LoadForecastser
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

In [ ]:
# 超参数
type_num = 10
batch_size = 24
n_features = 27
n_hidden = 128
seq_len = 96 * 7
n_layers = 2
out_features = 96
do = 0.2
lr = 0.0005
wd = 0.01
n_epochs = 150

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 导入要inverse transform的df数据,生成userID和coName的键值对
csv_path = './data/type_%s_csv_after_anomaly_detection/' % type_num
file_list = os.listdir(csv_path)
user_id_df_dict = {}
user_id_co_name_dict = {}
for i in range(len(file_list)):
    file = file_list[i]
    coName, userID = file[:-4].split('_')
    print(coName)
    df = pd.read_csv(csv_path + file)
    user_id_df_dict[userID] = df
    user_id_co_name_dict[userID] = coName

user_id_list = list(user_id_co_name_dict.keys())


In [ ]:
k = 7
userID = user_id_list[k]
coName = user_id_co_name_dict[userID]
print(coName)
user_df = user_id_df_dict[userID].copy()
col_load = user_df.loc[:, 'load'].values.reshape(-1, 1)

# 提取load的最大最小值
scaler = MinMaxScaler()
scaler = scaler.fit(col_load)
# del user_df
# del col_load
# del user_id_df_dict

In [ ]:
# 导入训练集和测试集

npy_path = './output/train_test/type%s/%s_%s/' % (type_num, coName, userID)

date_range = 365

if date_range == 90:
    test_x = np.load(npy_path + 'test_x_days_90.npy')
    test_y = np.load(npy_path + 'test_y_days_90.npy')
elif date_range == 365:
    test_x = np.load(npy_path + 'test_x_days_365.npy')
    test_y = np.load(npy_path + 'test_y_days_365.npy')



# numpy to tensor
test_x = torch.from_numpy(test_x).type(torch.Tensor)
test_y = torch.from_numpy(test_y).type(torch.Tensor)

model = LoadForecastser(n_features=n_features, n_hidden=n_hidden, seq_len=seq_len,
                        n_layers=n_layers, out_features=out_features,
                        do=do, device=device).to(device)
path = '/home/gavin/Documents/GitHub/FedML/fedml_experiments/standalone/fedavg/'
model.load_state_dict(torch.load(path + 'temp-20210428-030246.pth'))
model.eval()

# 模型用于测试集
test_x = test_x.to(device)
test_y_pred = model(test_x).to(device)

test_y_pred = test_y_pred.detach().cpu().numpy()

print(test_x.shape)

test_y = scaler.inverse_transform(test_y)
print(test_y.shape)

test_y_pred = scaler.inverse_transform(test_y_pred)
print(test_y_pred.shape)

testScore = math.sqrt(mean_squared_error(test_y[:, 0], test_y_pred[:, 0]))
print('Test Score: %.2f RMSE' % testScore)

In [ ]:
font = {'family': 'Times New Roman',
        'weight': 'normal',
        'size': 20,
       }
CB91_Blue = '#2CBDFE'
CB91_Green = '#47DBCD'
CB91_Pink = '#F3A0F2'
CB91_Purple = '#9D2EC5'
CB91_Violet = '#661D98'
CB91_Amber = '#F5B14C'
color_list = [CB91_Blue, CB91_Pink, CB91_Green, CB91_Amber,
              CB91_Purple, CB91_Violet]
n_fig = 24
fig = plt.figure()
fig.set_size_inches(20, 60, forward=True)
for i in range(n_fig):
    j = 96 * i
    ax = plt.subplot(n_fig, 1, i+1)
    y1 = test_y[j: j+96: 96, :]
    y1 = y1.reshape(y1.shape[0] * y1.shape[1])
    y2 = test_y_pred[j: j+96: 96, :]
    y2 = y2.reshape(y2.shape[0] * y2.shape[1])
    x = range(0, y1.shape[0])
    ax.plot(x, y1, color_list[0])
    ax.plot(x, y2, color_list[-1])
    plt.xticks(font=font)
    plt.yticks(font=font)
    ax.set_ylabel('values_day%s' % (i+1), font=font)
    ax.set_xticks(np.arange(0, y1.shape[0], 96))
    ax.set_xticklabels(np.arange(0, y1.shape[0]/96))

path = './output/img/testing/type%s/' % type_num
if not os.path.exists(path):
    os.makedirs(path)
plt.savefig(path + '/forward_per_day_%d_k_%d.jpg' % (int(testScore), k))
plt.show()

In [ ]:
i = 0
y1 = test_y[i::96, :]
y1 = y1.reshape(y1.shape[0] * y1.shape[1])
print(y1.shape)
y2 = test_y_pred[i::96, :]
y2 = y2.reshape(y2.shape[0] * y2.shape[1])
print(y2.shape)

font = {'family': 'Times New Roman',
        'weight': 'normal',
        'size': 20,
       }
CB91_Blue = '#2CBDFE'
CB91_Green = '#47DBCD'
CB91_Pink = '#F3A0F2'
CB91_Purple = '#9D2EC5'
CB91_Violet = '#661D98'
CB91_Amber = '#F5B14C'
color_list = [CB91_Blue, CB91_Pink, CB91_Green, CB91_Amber,
              CB91_Purple, CB91_Violet]

fig, ax = plt.subplots(figsize=(50, 10))
x = range(0, y1.shape[0])
ax.plot(x, y1, color_list[0])
ax.plot(x, y2, color_list[-1])
plt.xticks(font=font)
plt.yticks(font=font)
ax.set_ylabel('values', font=font)
ax.set_xticks(np.arange(0, y1.shape[0], 96))
ax.set_xticklabels(np.arange(0, y1.shape[0]/96))
plt.savefig('./output/img/forward_all_days_%d_k_%d.jpg' % (int(testScore), k))
plt.show()

In [28]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from daily_load_plotting import daily_load_plotting
import os
import shutil
from data_preprocessing.utils import find_files
import pandas as pd

from tsmoothie.smoother import *
from scipy import signal

In [29]:
def smoothing(df):
    data = df['load']
#     smoother = KalmanSmoother(component='level_longseason',
#                               component_noise={'level': 0.1, 'longseason': 0.1},
#                               n_longseasons=365)
#     data = smoother.smooth(data)
    b, a = signal.butter(8, 0.8, 'lowpass')   #配置滤波器 8 表示滤波器的阶数
    data = signal.filtfilt(b, a, data)  #data为要过滤的信号
    return data

In [30]:
def anomaly_detection(base_path, type_num, anomaly_detection_path, sum_flag=False, need_ad=True):
    type_num_original_path = base_path + 'data/type_%s/original/' % type_num
    type_num_after_anomaly_detection_path = base_path + 'data/type_%s/after_anomaly_detection/' % type_num

    sum_filename = 'type%s_%s' % (type_num, type_num)
    file_names_list = find_files(type_num_original_path)

    if sum_flag:
        print('Anomaly detection of sum file...')
        file_names_list = [sum_filename]
    else:
        print('Anomaly detection of single file...')
        try:
            file_names_list.remove(sum_filename)
        except:
            pass

    for file_name in file_names_list:
        co_name, user_id = file_name.split('_')
        print('-------' + co_name + '--------')
        print('-------' + user_id + '--------')

        # 导入行业x的数据
        df = pd.read_csv(type_num_original_path + file_name + '.csv')

        data = df.iloc[:, 1:].values
        print('Any nan?', np.any(np.isnan(data)))
        df = df.fillna(method='ffill')
        data = df.iloc[:, 1:].values
        print('Any nan?', np.any(np.isnan(data)))

        if need_ad:
            # 异常值分析 - 3σ原则
#             data = three_sigma_alg(anomaly_detection_path, df, user_id, co_name)
            data = smoothing(df)
        else:
            data = df['load']

        # 保存csv文件
        if not os.path.exists(type_num_after_anomaly_detection_path):
            os.makedirs(type_num_after_anomaly_detection_path)
        df['load'] = data
        df.to_csv(type_num_after_anomaly_detection_path + '%s_%s.csv' % (co_name, user_id), index=False)

In [31]:
base_path = '../../../Downloads/Thesis-temp/'
type_num = 7
anomaly_detection_path = base_path + 'output/img/type_%s/anomaly_detection/' % type_num
if os.path.exists(anomaly_detection_path):
    shutil.rmtree(anomaly_detection_path)
os.makedirs(anomaly_detection_path)
anomaly_detection(base_path, type_num, anomaly_detection_path, need_ad=True)
day_range = 48
week_range = 7
daily_load_plotting(base_path, type_num, start=1, end=150, week_range=week_range, day_range=day_range,
                    norm='standard')


Anomaly detection of single file...
-------南京第一农药集团有限公司(桠溪镇东风路8号)--------
-------150991350--------
Any nan? False
Any nan? False
-------南京先声东元制药有限公司--------
-------930131545--------
Any nan? False
Any nan? False
-------南京白敬宇制药有限责任公司--------
-------332212524--------
Any nan? False
Any nan? False

week: 1
week: 2
week: 3
week: 4
week: 5
week: 6
week: 7
week: 8
week: 9
week: 10
week: 11
week: 12
week: 13
week: 14
week: 15
week: 16
week: 17
week: 18
week: 19
week: 20
week: 21
week: 22
week: 1
week: 2
week: 3
week: 4
week: 5
week: 6
week: 7
week: 8
week: 9
week: 10
week: 11
week: 12
week: 13
week: 14
week: 15
week: 16
week: 17
week: 18
week: 19
week: 20
week: 21
week: 22
